# Building a new sensor
1. Define a specific sequence
2. Divide the sequence in windows of the same length of the trigger (36nt)
3. For each possible trigger, define a toehold sensor sequence
4. Filter sequences that contain stop codons
5. Evaluate sequence properties using NUPACK and rank them

### Import dependencies

In [2]:
import os
import os.path
from subprocess import Popen, PIPE
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from skbio import Sequence

### Define library functions

In [10]:
def reversed_complement(sequence):
    mapping = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}
    sequence_upper = sequence.upper()

    complement = ''
    for c in sequence_upper:
        complement += mapping[c]

    # reverse the sequence
    return complement[::-1]

def split_sequence(sequence, window):
    sequences = []
    limit = len(sequence) - window + 1

    for i in range(0, limit):
        sequences.append(sequence[i:window + i])

    return sequences

def no_stop(sequence):
    stop = ['UAA', 'UAG', 'UGA']

    for i in range(0, len(sequence), 3):
        if sequence[i:i + 3] in stop:
            return False

    return True

def possible_toehold_B(reg, rev):
    loop = 'GGACUUUAGAACAGAGGAGAUAAAGAUG'

    #What is the RBS ?
    linker = 'ACCTGGCGGCAGCGCAAGAAGA'#( from Green paper 2019 se, modified with an extra A for creating C site MOCLO)
    #linker = "AACCUGGCGGCAGCGCAAGAAGAUGCGUAAA" 
    toeholds = []

    #for n in ['A', 'G', 'U', 'C']:
    if no_stop(reg[0:11] + linker):
            toeholds.append(rev + loop + reg[0:11] + linker)
            return toeholds
      
    
    toeholds.append("STOP")
    return toeholds

### Step 1: Define a target sequence
In this case, we use the  5' of the glycoprotein 1 from PVY:

In [4]:
seq = "ATGGCAACTTACATGTCAACAATCTGTTTTGGTTCGTTTGAATGCAAGCTACCATACTCACCAGCCTCTTGCGAGCATATTGTGAAGGAACGAGAAGTGCCGGCTTCCGTTGATCCTTTCGCAGATCTGGAAACACAACTTAGTGCACGATTGCTCAAGCAAAAATATGCTACTGTTCGTGTGCTCAAAAACGGTACTTTTACGTACCGATACAAGACTGATGCCCAGATAATGCGCATTCAGAAGAAACTGGAGAGGAAGGATAGGGAAGAATATCACTTCCAAATGGCCGCTCCTAGTATTGTGTCAAAAATTACTATAGCTGGCGGAGATCCTCCATCAAAGTCTGAGCCACAAGCACCAAGAGGGATCATTCATACAACTCCAAGGATGCGTAAAGTCAAGACACGCCCCATAATAAAGTTGACAGAAGGCCAGATGAATCACCTCATTAAGCAGATAAAACAGATTATGTCGGAGAAAAGAGGGTCTGTCCACTTAATTAGTAAGAAAACCACTCATGTTCAATATAAGAAGATACTTGGTGCATACTCCGCAGCGGTTCGAACTGCACATATGATGGGTTTGCGACGGAGAGTGGACTTCCGATGTGATATGTGGACAGTTGGACTTTTGCAACGTCTCGCTCGGACGGACAAATGGTCCAATCAAGTCCGCACTATCAACATACGAAGGGGTGATAGTGGAGTCATCTTGAACACAAAAAGCCTCAAAGGCCACTTTGGTAGAAGTTCAGGAGGCTTGTTCATAGTGCGTGGATCACACGAAGGGAAATTGTATGATGCACGTTCTAGAGTTACTCAGAGTATTTTAAACTCAATGATCCAGTTTTCGAATGCCGACAATTTTTGGAAGGGTCTGGACGGTAATTGGGCACGAATGAGATATCCTTCGGATCACACATGTGTAGCTGGTTTACCTGTCGAAGATTGTGGTAGGGTAGCTGCATTGATGGCACACAGTATCCTTCCGTGCTATAAGATAACTTGCCCCACCTGTGCTCAACAGTATGCCAGCTTGCCAGTTAGC"

Convert to RNA and determine the reverse complement

In [6]:
processed_sequence = seq.upper().replace('T', 'U') #/.replace(' ', '')
rc = reversed_complement(processed_sequence)
len(split_sequence(rc,36))

### Step 2: Determine 36-nucleotide sub-sequences
To do this, we make all possible triggers for the direct and reversed complementary sequence

In [13]:
d_1 = {'Triggers': split_sequence(processed_sequence,36)}
df_1 = pd.DataFrame(data=d_1)
df_1["Sense"]="Direct"

d_2 = {'Triggers': split_sequence(rc,36)}
df_2 = pd.DataFrame(data=d_2)
df_2["Sense"]="Reversed Complement"
frames = [df_1, df_2]
result = pd.concat(frames)
#result

### Step 3: For each trigger, design a toehold sensor

In [14]:
toeholds=[]
for i in range(len(result.iloc[:,0])):
    toeholds.append((possible_toehold_B(result.iloc[i,0],reversed_complement(result.iloc[i,0])))[0])
a=pd.Series(toeholds)
result["Toehold Switch"]=a.values
#result

### Step 4: Remove sensors with STOP codons

In [19]:
df = result[result.iloc[:,2] != "STOP"]
#df

### Step 5: Evaluate sequence properties using NUPACK 

Scoring functions from Green (2019):
- Three-parameter fit (R2 = 0.57):
- Fold change = –71.7 dfull_sensor  – 49.1 dactive_sensor – 22.6 dbinding_site + 54.3
- Four-parameter fit (R2 = 0.60):
- Fold change = –93.2 dfull_sensor – 43.3 dactive_sensor – 22.1 dbinding_site – 9.4 dmin_target + 61.3

### Library functions
- Calculation of the minimum free energy (MFE) secondary structure of a singular RNA sequence
- NUPACK analysis

In [12]:
def DG(sequence,result_path,wait):
    file = open('{}pipo.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.close()
    final=[]
    semi_final=[]

    Popen(["mfe -T 29 {}pipo".format(result_path)],shell=True, stdout=PIPE)
    time.sleep(wait)
    lenght=len(sequence)
    with open("{}pipo.mfe".format(result_path)) as res:
        for r in res:
            r = r.strip('\n')
            if not r.startswith('%'):
                r = r.split('\t')
                semi_final.append(r)

    #final.append()                

    return (float(semi_final[2][0]))

    os.remove("{}pipo.mfe".format(result_path,))
    os.remove("{}pipo.in".format(result_path))

def nupack_analysis(sequence, secondary_sensor,  window, sensor_type, result_path):
    list_for_table = []

    processed_sequence = sequence.upper().replace('T', 'U').replace(' ', '')
    reg_sequences = split_sequence(processed_sequence, window)
    rev_comp_sequences = [reversed_complement(s) for s in reg_sequences]

    if sensor_type == 'A':
        target_toehold_map = possible_toehold_A(reg_sequences, rev_comp_sequences)
    else:
        target_toehold_map = possible_toehold_B(reg_sequences, rev_comp_sequences)

    sequence = sequence.upper().replace('T', 'U')
    single_streadness_sequence = single_streadness(sequence, result_path, wait=6)
    for target, toehold in target_toehold_map.items():
        id = sequence.index(target)

        target_defect = sum(single_streadness_sequence[id:id+36])/36
        toehold_defect = sum(single_streadness(toehold, result_path)[0:30])/30
        sensor_defect = complex_defect(toehold, secondary_sensor, result_path)

        score = 5*(1-target_defect) + 4*(1-toehold_defect) + 3*sensor_defect

        list_for_table.append(tuple([target[0:36], toehold, 1-target_defect, 1-toehold_defect, sensor_defect, score]))

    return list_for_table

Structure of the designed optimal interaction betwen a Sensor and its trigger

In [17]:
secondary_2=".......................................(((((............)))))....................................+...................................."

In [18]:
secondary_sensor_B = '.........................(((((((((((...(((((............)))))...)))))))))))......................'